In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import readVCF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
vcfpath = '/big/results/bsm/2020-06-10-chromatin-state/MSSM_109_brain.ploidy_50.filtered-chromatin.vcf'

In [3]:
%connect_info

{
  "shell_port": 56699,
  "iopub_port": 34851,
  "stdin_port": 45119,
  "control_port": 53065,
  "hb_port": 43279,
  "ip": "127.0.0.1",
  "key": "938f6a82-3a619dd63b915133d13a5a67",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-ce3b72e0-cf36-41a5-8482-dd1b67032e40.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
